# Build a RAG solution using Azure Cosmos DB

Steps in this notebook:
1. Setup Azure Cosmos DB - database, container, policies (vector embedding, full text search, and indexing)
2. Create embeddings
3. Upload data to the container
4. Send a query to the search engine to check results
5. Send query results to a language model to generate response

Note: Steps 1-4: Done during initial setup

## Install Dependencies

In [ ]:
%pip install numpy
%pip install openai
%pip install python-dotenv
%pip install azure-core
%pip install azure-cosmos

## Azure Resources Needed
1. Azure Cosmos DB
    - Full-Text & Hybrid Search for NoSQL API (preview) set to On
    - Vector Search for NoSQL API set to On

2. Azure OpenAI
    - Deploy GPT-4o, text-embedding-3-large


## Load Azure configurations

You always need to run this!

In [1]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_openai_embedding_size = 1536

azure_cosmosdb_endpoint = os.getenv("AZURE_COSMOSDB_ENDPOINT")
azure_cosmosdb_key = os.getenv("AZURE_COSMOSDB_KEY")
azure_cosmosdb_database = "azureservicesdatabase"
azure_cosmosdb_container = "azureservicescontainer"

## Setup Azure Cosmos DB

In [2]:
from azure.cosmos import CosmosClient
from azure.cosmos import PartitionKey, exceptions

# Setup the connection
cosmos_client = CosmosClient(url=azure_cosmosdb_endpoint, credential=azure_cosmosdb_key)

# Create database
db = cosmos_client.create_database_if_not_exists(id=azure_cosmosdb_database)

# Author the vector embedding policy
vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path":"/titleVector",
            "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions":azure_openai_embedding_size
        },
        {
            "path":"/contentVector",
            "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions":azure_openai_embedding_size
        }
    ]
}

full_text_policy = {
   "defaultLanguage": "en-US",
   "fullTextPaths": [
       {
           "path": "/title",
           "language": "en-US"
       },
       {
           "path": "/content",
           "language": "en-US"
       },
       {
           "path": "/category",
           "language": "en-US"
       }
   ]
}

# Add vector indexes to indexing policy
indexing_policy = {
    "includedPaths": [
        {
            "path": "/*"
        }
    ],
    "excludedPaths": [
        {
            "path": "/\"_etag\"/?"
        },
        {
            "path": "/titleVector/*"
        },
        {
            "path": "/contentVector/*"
        }
    ],
    "fullTextIndexes": [
        {
            "path": "/title"
        },
        {
            "path": "/content"
        },
        {
            "path": "/category"
        }
    ],
    "vectorIndexes": [
        {"path": "/titleVector",
         "type": "quantizedFlat"
        },
        {"path": "/contentVector",
         "type": "quantizedFlat"
        }
    ]
}

try:    
    container = db.create_container_if_not_exists(
                    id=azure_cosmosdb_container,
                    partition_key=PartitionKey(path='/id', kind='Hash'),
                    indexing_policy=indexing_policy,
                    vector_embedding_policy=vector_embedding_policy,
                    full_text_policy=full_text_policy)

    print('Container with id \'{0}\' created'.format(id))

except exceptions.CosmosResourceExistsError:
    print('A container with id \'{0}\' already exists'.format(id))

container = db.get_container_client(azure_cosmosdb_container)

Container with id '<built-in function id>' created


## Creating embeddings separately

We are computing the embeddings manually

In [3]:
from openai import AzureOpenAI
import json

# Azure OpenAI client
openai_client = AzureOpenAI(
    # to get version: https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    azure_deployment=azure_openai_embeddings_deployment,
    api_key=azure_openai_key)

# Read the AzureServices.json
path = os.path.join('Data/azureservices/', 'AzureServices.json')
with open(path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

titles = [item['title'] for item in input_data]
content = [item['content'] for item in input_data]
title_response = openai_client.embeddings.create(input=titles, 
                                                 model=azure_openai_embeddings_deployment, 
                                                 dimensions=azure_openai_embedding_size)
title_embeddings = [item.embedding for item in title_response.data]
content_response = openai_client.embeddings.create(input=content, 
                                                   model=azure_openai_embeddings_deployment, 
                                                   dimensions=azure_openai_embedding_size)
content_embeddings = [item.embedding for item in content_response.data]

# Generate embeddings for title and content fields
for i, item in enumerate(input_data):
    title = item['title']
    content = item['content']
    item['titleVector'] = title_embeddings[i]
    item['contentVector'] = content_embeddings[i]

# Output embeddings to json file
output_path = os.path.join('Data/azureservices', 'AzureServicesVectors.json')
output_directory = os.path.dirname(output_path)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
with open(output_path, "w") as f:
    json.dump(input_data, f)

## Upload data to the container

In [4]:
import json

with open('Data/azureservices/AzureServicesVectors.json') as f:
   data = json.load(f)

container_client = db.get_container_client(azure_cosmosdb_container)

for item in data:
    print("writing item ", item['id'])
    container_client.upsert_item(item)

writing item  1
writing item  2
writing item  3
writing item  4
writing item  5
writing item  6
writing item  7
writing item  8
writing item  9
writing item  10
writing item  11
writing item  12
writing item  13
writing item  14
writing item  15
writing item  16
writing item  17
writing item  18
writing item  19
writing item  20
writing item  21
writing item  22
writing item  23
writing item  24
writing item  25
writing item  26
writing item  27
writing item  28
writing item  29
writing item  30
writing item  31
writing item  32
writing item  33
writing item  34
writing item  35
writing item  36
writing item  37
writing item  38
writing item  39
writing item  40
writing item  41
writing item  42
writing item  43
writing item  44
writing item  45
writing item  46
writing item  47
writing item  48
writing item  49
writing item  50
writing item  51
writing item  52
writing item  53
writing item  54
writing item  55
writing item  56
writing item  57
writing item  58
writing item  59
writin

## Vector Search helper function

In [5]:
from openai import AzureOpenAI
from azure.cosmos import CosmosClient

# Simple function to assist with vector search
def vector_search(user_query, num_results):

    # Setup the connection
    cosmos_client = CosmosClient(url=azure_cosmosdb_endpoint, credential=azure_cosmosdb_key)
    database = cosmos_client.get_database_client(azure_cosmosdb_database)
    container = database.get_container_client(azure_cosmosdb_container)

    # Azure OpenAI client
    openai_client = AzureOpenAI(
        api_version=azure_openai_api_version,
        azure_endpoint=azure_openai_endpoint,
        azure_deployment=azure_openai_embeddings_deployment,
        api_key=azure_openai_key
    )

    response = openai_client.embeddings.create(input=user_query, 
                                               model=azure_openai_embeddings_deployment, 
                                               dimensions=azure_openai_embedding_size)
    embedding = response.data[0].embedding

    results = container.query_items(
            query='''
                SELECT TOP @num_results c.id, c.title, c.content, c.category, VectorDistance(c.contentVector,@embedding) AS SimilarityScore 
                FROM c 
                ORDER BY VectorDistance(c.contentVector, @embedding)
            ''',
            parameters=[
                {"name": "@embedding", "value": embedding}, 
                {"name": "@num_results", "value": num_results} 
            ],
            enable_cross_partition_query=True)

    # Extract the necessary information from the results
    formatted_results = []
    for document in results:
        score = document.pop('SimilarityScore')
        formatted_result = {
            'SimilarityScore': score,
            'document': document
        }
        formatted_results.append(formatted_result)

    return formatted_results    
    

## Vector Query Search

In [6]:
query = "What is Azure Firewall?"
results = vector_search(query, 5)

for document in results:
        print(f"Similarity Score: {document['SimilarityScore']}")
        print(f"ID: {document['document']['id']}")
        print(f"Title: {document['document']['title']}")
        print(f"Category: {document['document']['category']}")
        print(f"Content: {document['document']['content']}\n")

Similarity Score: 0.8003057330233618
ID: 24
Title: Azure Firewall
Category: Security
Content: Azure Firewall is a managed, cloud-based network security service that protects your Azure Virtual Network resources. It provides features like stateful packet inspection, application filtering, and threat intelligence. Firewall supports various network protocols, including TCP, UDP, and ICMP. You can use Azure Firewall to create and enforce network security policies, prevent unauthorized access, and protect your applications and data. It also integrates with other Azure services, such as Azure Monitor and Azure Security Center.

Similarity Score: 0.5747706539905616
ID: 44
Title: Azure Application Gateway
Category: Networking
Content: Azure Application Gateway is a fully managed, scalable, and secure web application firewall (WAF) that helps protect your web applications from common web vulnerabilities and exploits. It provides features like SSL offloading, URL-based routing, and custom health

## Send query results to a language model to generate response

In [7]:
from openai import AzureOpenAI

# Azure OpenAI client
openai_client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key)

# Provide instructions to the model
SYSTEM_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# User Query
query = "What is Azure Firewall?"

results = vector_search(query, 5)

# Use a unique separator to make the sources distinct. 
# We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
sources_formatted = "=================\n".join([f"TITLE: {document['document']['title']}, CONTENT: {document['document']['content']}, CATEGORY: {document['document']['category']}" for document in results])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": SYSTEM_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=azure_openai_deployment
)

print(response.choices[0].message.content)


Azure Firewall is a managed, cloud-based network security service that protects your Azure Virtual Network resources. It offers features like stateful packet inspection, application filtering, and threat intelligence. Azure Firewall supports various network protocols, including TCP, UDP, and ICMP. It allows you to create and enforce network security policies, prevent unauthorized access, and protect your applications and data. Additionally, it integrates with other Azure services, such as Azure Monitor and Azure Security Center. [SOURCE: Azure Firewall]


## Full text search

In [13]:
from azure.cosmos import CosmosClient
import json

# Simple function to assist with full text search
def full_text_search(user_query, num_results):

    # Setup the connection
    cosmos_client = CosmosClient(url=azure_cosmosdb_endpoint, credential=azure_cosmosdb_key)
    database = cosmos_client.get_database_client(azure_cosmosdb_database)
    container = container = database.get_container_client(azure_cosmosdb_container)

    # Build the query with str.format() method
    query = '''
        SELECT TOP {0} c.id, c.title, c.category, c.content
        FROM c
        WHERE FullTextContains(c.title, '{1}')
    '''.format(num_results, user_query)

    results = container.query_items(
            query=query,
            enable_cross_partition_query=True)

    items = [item for item in results]
    
    output = json.dumps(items, indent=True)
    print(output)
      
full_text_search("Firewall", 5)

[
 {
  "id": "24",
  "title": "Azure Firewall",
  "category": "Security",
  "content": "Azure Firewall is a managed, cloud-based network security service that protects your Azure Virtual Network resources. It provides features like stateful packet inspection, application filtering, and threat intelligence. Firewall supports various network protocols, including TCP, UDP, and ICMP. You can use Azure Firewall to create and enforce network security policies, prevent unauthorized access, and protect your applications and data. It also integrates with other Azure services, such as Azure Monitor and Azure Security Center."
 }
]


## Hybrid Search

In [14]:
from openai import AzureOpenAI
from azure.cosmos import CosmosClient
import json

# Simple function to assist with full text search
def hybrid_search(user_query, num_results):

    # Setup the connection
    cosmos_client = CosmosClient(url=azure_cosmosdb_endpoint, credential=azure_cosmosdb_key)
    database = cosmos_client.get_database_client(azure_cosmosdb_database)
    container = container = database.get_container_client(azure_cosmosdb_container)

    # Azure OpenAI client
    openai_client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    azure_deployment=azure_openai_embeddings_deployment,
    api_key=azure_openai_key)

    response = openai_client.embeddings.create(input=user_query, 
                                               model=azure_openai_embeddings_deployment, 
                                               dimensions=1536)
    embedding = response.data[0].embedding


    # Build the query with str.format() method
    query = '''
        SELECT TOP {0} c.id, c.title, c.category, c.content
        FROM c
        ORDER BY RANK RRF 
            (VectorDistance(c.contentVector, {1}), FullTextScore(c.title, ['{2}']))
    '''.format(num_results, embedding, user_query)

    results = container.query_items(
            query=query,
            enable_cross_partition_query=True)

    items = [item for item in results]
    
    output = json.dumps(items, indent=True)
    print(output)
      
hybrid_search("what is azure firewall", 5)

[
 {
  "id": "24",
  "title": "Azure Firewall",
  "category": "Security",
  "content": "Azure Firewall is a managed, cloud-based network security service that protects your Azure Virtual Network resources. It provides features like stateful packet inspection, application filtering, and threat intelligence. Firewall supports various network protocols, including TCP, UDP, and ICMP. You can use Azure Firewall to create and enforce network security policies, prevent unauthorized access, and protect your applications and data. It also integrates with other Azure services, such as Azure Monitor and Azure Security Center."
 },
 {
  "id": "44",
  "title": "Azure Application Gateway",
  "category": "Networking",
  "content": "Azure Application Gateway is a fully managed, scalable, and secure web application firewall (WAF) that helps protect your web applications from common web vulnerabilities and exploits. It provides features like SSL offloading, URL-based routing, and custom health probes. A